# Clone Portal users, groups and content

This sample notebook can be used for cloning a portal, from say, a staging to a production environment. It clones the users, groups and the content. It does not copy over services and data though, and works at the tier of portal items.

In [1]:
from arcgis.gis import GIS
from IPython.display import display
from getpass import getpass

## Define the source and target portals
To start with, define the source and target portals. Connect to them using accounts with administrative privileges:

In [2]:
source_password = getpass()
target_password = getpass()
source = GIS("https://dev003327.esri.com/portal", "admin", source_password, verify_cert=False)
target = GIS("https://dev003735.esri.com/portal", "admin", target_password)
target_admin_username = 'admin'

········
········


# Users
List the users in the source and target portals:

In [6]:
#!esri_ & !admin
source_users = source.users.search('!esri_ & !admin')
source_users

[<User username:adams.powell>,
 <User username:allen.price>,
 <User username:anderson.bailey>,
 <User username:baker.long>,
 <User username:brown.rogers>,
 <User username:campbell.bryant>,
 <User username:carter.flores>,
 <User username:clark.ramirez>,
 <User username:davis.reed>,
 <User username:edwards.griffin>,
 <User username:evans.russell>,
 <User username:garcia.torres>,
 <User username:gonzalez.patterson>,
 <User username:green.perry>,
 <User username:hall.sanders>,
 <User username:harris.cox>,
 <User username:hernandez.wood>,
 <User username:hill.coleman>,
 <User username:jackson.cooper>,
 <User username:johnson.stewart>,
 <User username:jones.morris>,
 <User username:king.barnes>,
 <User username:lee.brooks>,
 <User username:lewis.watson>,
 <User username:lopez.henderson>,
 <User username:martin.howard>,
 <User username:martinez.peterson>,
 <User username:miller.cook>,
 <User username:mitchell.washington>,
 <User username:moore.bell>,
 <User username:nelson.hughes>,
 <User use

In [7]:
len(source_users)

49

### Limit number of users for demo purpose

In [8]:
source_users = source_users[:5]
len(source_users)

5

In [9]:
target_users = target.users.search()
target_users

[<User username:admin>,
 <User username:arcgis_python_api>,
 <User username:esri_boundaries>,
 <User username:esri_demographics>,
 <User username:esri_livingatlas>,
 <User username:esri_nav>,
 <User username:publisher1>,
 <User username:system_publisher>]

If source users are already in the target, run the following code to delete them:

In [10]:
# create a list of system accounts that should not be modified
systemusers = ['system_publisher', 'esri_nav', 'esri_livingatlas', 
               'esri_boundaries', 'esri_demographics']

### Remove existing users from target portal
Assign their content to admin account and delete the account

for srcuser in source_users:
    if not srcuser.username in system_users:
        #don't delete the account used to connect
        if srcuser.username != target_admin_username:
            try:
                targetusr = target.users.get(srcuser.username)
                if targetusr is not None:
                    print('Deleting user: ' + targetusr.fullName)
                    targetusr.reassign_to(target_admin_username)
                    targetusr.delete()
            except:
                print('User {} does not exist in Target Portal'.format(srcuser.username))

### Copy Users
Create a function that will accept connection to the target portal, user objects and password to create users with.

In [11]:
su1 = source_users[0]
su1

<User username:adams.powell>

In [13]:
su1.__dict__.keys()

dict_keys(['storageQuota', 'preferredView', 'description', 'modified', 'email', 'mfaEnabled', 'fullName', 'username', 'tags', 'provider', '_workdir', 'lastName', 'groups', 'lastLogin', 'thumbnail', 'region', 'created', 'level', 'units', 'role', '_hydrated', 'orgId', 'storageUsage', '_portal', 'validateUserProfile', 'culture', 'privileges', 'firstName', '_gis', 'userType', 'idpUsername', 'access', 'favGroupId', 'disabled'])

In [14]:
def copy_user(target_portal, source_user, password):
    # See if the user has firstName and lastName properties
    try:
        first_name = source_user.firstName
        last_name = source_user.lastName
    except:
        # if not, split the fullName
        full_name = source_user.fullName
        first_name = full_name.split()[0]
        try:
            last_name = full_name.split()[1]
        except:
            last_name = 'NoLastName'

    try:
        # create user
        target_user = target_portal.users.create(source_user.username, password, first_name, last_name,
                                          source_user.email, source_user.description, source_user.role)

        # update user properties
        target_user.update(source_user.access, source_user.preferredView,
                           source_user.description, source_user.tags, source_user.get_thumbnail_link(),
                           culture=source_user.culture, region=source_user.region)
        return target_user
    
    except Exception as Ex:
        print(str(Ex))
        print("Unable to create user "+ source_user.username)
        return None

For each user in source portal, make a corresponding user in target portal:

In [15]:
for user in source_users:
    print("Creating user: " + user.username)
    copy_user(target, user, 'TestPassword@123')

Creating user: adams.powell
Creating user: allen.price
Creating user: anderson.bailey
Creating user: baker.long
Creating user: brown.rogers


Verify that users have been added to target portal:

In [16]:
target_users = target.users.search()
target_users

[<User username:adams.powell>,
 <User username:admin>,
 <User username:allen.price>,
 <User username:anderson.bailey>,
 <User username:arcgis_python_api>,
 <User username:baker.long>,
 <User username:brown.rogers>,
 <User username:esri_boundaries>,
 <User username:esri_demographics>,
 <User username:esri_livingatlas>,
 <User username:esri_nav>,
 <User username:publisher1>,
 <User username:system_publisher>]

Thus users have been successfully added to the target portal

# Groups

List the groups in the source and target portals:

In [17]:
#source_groups = source.groups.search()
source_groups = source.groups.search("!owner:esri_* & !Basemaps")
source_groups

[<Group title:"Central Services" owner:admin>,
 <Group title:"Compliance" owner:admin>,
 <Group title:"Customer Service, Finance, Billing and Accounting" owner:admin>,
 <Group title:"Demographic Content" owner:admin>]

In [18]:
#target_groups = target.groups.search()
target_groups = target.groups.search("!owner:esri_*")
target_groups

[<Group title:"Featured Maps and Apps" owner:admin>]

If source groups are already in the target, run the following code to delete them. If the group belongs to any of built-in user accounts, don't delete it.

for tg in target_groups:
    for sg in source_groups:
        if sg.title == tg.title and (not tg.owner in systemusers):
            print("Cleaning up group {} in target Portal...".format(tg.title))
            tg.delete()
            break

## Copy Groups

In [19]:
import tempfile

GROUP_COPY_PROPERTIES = ['title', 'description', 'tags', 'snippet', 'phone',
                         'access', 'isInvitationOnly']

def copy_group(target, source, group):
    """ Copy group to the target portal."""
    with tempfile.TemporaryDirectory() as temp_dir:
        # Create new groups with the subset of properties we want to
        # copy to the target portal. Handle switching between org and
        # public access when going from an org in a multitenant portal
        # and a single tenant portal
        target_group = {}
        
        for property_name in GROUP_COPY_PROPERTIES:
            target_group[property_name] = group[property_name]

        if target_group['access'] == 'org' and target.properties['portalMode'] == 'singletenant':
            target_group['access'] = 'public'
        elif target_group['access'] == 'public'\
             and source.properties['portalMode'] == 'singletenant'\
             and target.properties['portalMode'] == 'multitenant'\
             and 'id' in target.properties: # is org
            target_group['access'] = 'org'

        # Handle the thumbnail (if one exists)
        thumbnail_file = None
        if 'thumbnail' in group:
            target_group['thumbnail'] = group.download_thumbnail(temp_dir)

        # Create the group in the target portal
        copied_group = target.groups.create_from_dict(target_group)
        
         # Reassign all groups to correct owners, add users, and find shared items
        members = group.get_members()
        if not members['owner'] == target_admin_username:
            copied_group.reassign_to(members['owner'])
        if members['users']:
            copied_group.add_users(members['users'])
        return copied_group

For each group in source portal, make a corresponding group in target portal and a create a dictionary of source group id and corresponding group id in target portal:

In [20]:
copied_groups = {}
for group in source_groups:
    print("Copying group: " + group.title)
    tgt_group = copy_group(target, source, group)
    copied_groups[group.groupid] = tgt_group.groupid

Copying group: Central Services
Copying group: Compliance
Copying group: Customer Service, Finance, Billing and Accounting
Copying group: Demographic Content


Verify that groups have been created in the target portal:

In [21]:
target_groups = target.groups.search()
target_groups

[<Group title:"Central Services" owner:admin>,
 <Group title:"Compliance" owner:admin>,
 <Group title:"Customer Service, Finance, Billing and Accounting" owner:admin>,
 <Group title:"Demographic Content" owner:admin>,
 <Group title:"Esri Boundary Layers" owner:esri_boundaries>,
 <Group title:"Esri Demographic Layers" owner:esri_demographics>,
 <Group title:"Featured Maps and Apps" owner:admin>,
 <Group title:"Living Atlas" owner:esri_livingatlas>,
 <Group title:"Living Atlas Analysis Layers" owner:esri_livingatlas>,
 <Group title:"Navigator Maps" owner:esri_nav>]

Print the mapping of source and target group ids:

In [22]:
copied_groups

{'0558325eac9942b6aaa3453016ecc91c': '1d07937d440d4cf598ca2eedb3027482',
 '1a50448b8d3747a0bb7a4fcf821879d6': 'aefcd77d89f64112b0243e8b2518aa87',
 '6fcb60c6581e4aef86a34adeaa42f90c': '2567bcde140e495db40be4f4a3255351',
 '7a9ea7b178ed4c498455ea36f9281146': 'b73f27e1b8c5454aaa57c4dbe7c3ad3f'}

With this part of the sample, we have successfully created users, groups and added the appropriate users to these groups.

In [23]:
group1 = target_groups[0]
group1.get_members()

{'admins': ['admin'],
 'owner': 'admin',
 'users': ['baker.long', 'brown.rogers', 'adams.powell', 'anderson.bailey']}

# Items

Copying items consists of multiple steps. The following section of the sample does the following

 1. Create a dictionary of itemIds and `Item` objects for each user in each folder
 2. Prepare sharing information for each item

## Create a dictionary of itemIds and item objects
Do this for each user and each folder in the user account

In [24]:
source_items_by_id = {}
for user in source_users:
    print("Collecting item ids for {}...".format(user.username))
    user_content = user.items()
    # Copy item ids from root folder first
    for item in user_content:
        source_items_by_id[item.itemid] = item 
    # Copy item ids from folders next
    folders = user.folders
    for folder in folders:
        folder_items = user.items(folder=folder['title'])
        for item in folder_items:
            source_items_by_id[item.itemid] = item

In [25]:
source_items_by_id

{'332b3e2370b74ba3a2a5d3893f4dd904': <Item title:"Allen Price response locations" type:Web Map owner:allen.price>,
 '3e7b1b056b124fb0b48398bdfe36577d': <Item title:"Brown Rogers response locations" type:Web Map owner:brown.rogers>,
 '3ff2216d367244b99578a710d97d2be7': <Item title:"OH" type:Feature Service owner:allen.price>,
 '5b2d580eff584fa597b2eb0d8b2d7498': <Item title:"OH" type:CSV owner:allen.price>,
 '8502a55fd04e489a8c5f43c303f46be6': <Item title:"WY" type:Feature Service owner:baker.long>,
 '859405481aad42bc89f0f258f8534cca': <Item title:"SC" type:Feature Service owner:adams.powell>,
 'aff676230201458ebb679bc0ac35e70e': <Item title:"AR" type:CSV owner:brown.rogers>,
 'b3e690006a964cceb0c7b0aced1ad283': <Item title:"Baker Long response locations" type:Web Map owner:baker.long>,
 'bd7062097d234088af4194c35b540a50': <Item title:"AR" type:Feature Service owner:brown.rogers>,
 'ebeedb8a91d642e6b1248d6c625e1101': <Item title:"Adams Powell response locations" type:Web Map owner:adams

## Prepare sharing information for each item

In [26]:
# Remap the group id of items to point to new group id
for group in source_groups:
    target_group_id = copied_groups[group.groupid]
    for group_item in group.content():
        try:
            item = source_items_by_id[group_item.itemid]
            if item is not None:
                if not 'groups'in item:
                    item['groups'] = []
                item['groups'].append(target_group_id)
        except:
            print("Not found item : " + group_item.itemid)

### Print a mapping of item and its group membership

In [27]:
for key in source_items_by_id.keys():
    item = source_items_by_id[key]
    print("\n{:40s}".format(item.title), end = " # ")
    if 'groups' in item:
        print(item.access, end = " # ")
        print(item.groups, end = "")


SC                                       # 
OH                                       # 
Allen Price response locations           # 
WY                                       # 
SC                                       # 
Adams Powell response locations          # 
Brown Rogers response locations          # 
Baker Long response locations            # 
AR                                       # 
OH                                       # 
WY                                       # 
AR                                       # 

## Copy Items

In [28]:
TEXT_BASED_ITEM_TYPES = frozenset(['Web Map', 'Feature Service', 'Map Service','Web Scene',
                                   'Image Service', 'Feature Collection', 
                                   'Feature Collection Template',
                                   'Web Mapping Application', 'Mobile Application', 
                                   'Symbol Set', 'Color Set',
                                   'Windows Viewer Configuration'])
ITEM_COPY_PROPERTIES = ['title', 'type', 'typeKeywords', 'description', 'tags',
                        'snippet', 'extent', 'spatialReference', 'name',
                        'accessInformation', 'licenseInfo', 'culture', 'url', ]

def copy_item(target, owner, folder, item):
    with tempfile.TemporaryDirectory() as temp_dir:
        copy_item = {}
        for property_name in ITEM_COPY_PROPERTIES:
            copy_item[property_name] = item[property_name]

        data_file = None
        if item.type in TEXT_BASED_ITEM_TYPES:
            # If its a text-based item, then read the text and add it to the request.
            if item.size > 0:
                text = item.get_data(False)
                #textstr = text.decode('utf-8')
                copy_item['text'] = text
        elif item.size > 0: # download data for all other types, not just item.type in FILE_BASED_ITEM_TYPES:
            # download data and add to the request as a file
            data_file = item.download(temp_dir)

        thumbnail_file = item.download_thumbnail(temp_dir)

        metadata_file = item.download_metadata(temp_dir)

        # Add the item to the target portal
        copied_item = target.content.add(copy_item, data_file, thumbnail_file, 
                                         metadata_file, owner, folder)

        return copied_item

In [29]:
RELATIONSHIP_TYPES = frozenset(['Map2Service', 'WMA2Code',
                                'Map2FeatureCollection', 'MobileApp2Code', 'Service2Data',
                                'Service2Service'])

def copy_relationships(target, copied_items, src_item, relationships, owner, folder):
    
    target_item_id = copied_items.get(src_item.itemid)
    if target_item_id is not None:
        target_item = target.content.get(target_item_id)

        for rel_type in RELATIONSHIP_TYPES:
            src_rel_items = src_item.related_items(rel_type)

            for src_rel_item in src_rel_items:
                print("***Found related items for " + src_rel_item.title)
                source_rel_id = src_rel_item.itemid

                # See if it's already been copied to the target
                target_rel_id = copied_items.get(source_rel_id)
                if not target_rel_id:
                    # If not, then copy it to the target - folder may have moved though?
                    target_rel_item = clone_item(target, owner, folder, src_rel_item)

                    if target_rel_item is not None:
                        # add relationship from target_item to copied item
                        result = target_item.add_relationship(target_rel_item, rel_type)

                        if not result:
                            print('Unable to add relationship from ' +  target_item.itemid + ' to ' + target_rel_item.itemid)
                    else:
                        print("@@@Error Cloning Item "+src_rel_item.title)

In [30]:
copied_items = {}
relationships = RELATIONSHIP_TYPES

for user in source_users:
    #if not user.username in systemusers:
    print("**************\n"+user.username)
    usercontent = user.items()
    folders = user.folders
    for item in user_content:
        try:
            copied_item = copy_item(target, user, None, item)
            if copied_item is not None:
                copied_items[item.itemid] = copied_item.itemid
                # share the item
                copied_item.share(item.access == 'public',
                                    item.access in ['org', 'public'],
                                    source_items_by_id[item.itemid].groups 
                                        if 'groups' in source_items_by_id[item.itemid]
                                        else None)
                display(item)
            else:
                print('Error copying ' + item.title)
        except:
            print("Error copying " + item.title)

        for folder in folders:
            target.content.create_folder(folder, user)
            folder_items = user.items(folder['title'])
            for item in folderitems:
                try:
                    copied_item = copy_item(target, user, folder['title'], item)
                    if copied_item is not None:
                        copied_items[item.itemid] = copied_item.itemid

                        # share the item
                        copied_item.share(item.access == 'public',
                                          item.access in ['org', 'public'],
                                          source_items_by_id[item.itemid].groups 
                                              if 'groups' in source_items_by_id[item.itemid]
                                              else None)
                    else:
                        print('Error copying ' + item.title)
                except:
                    print("Error copying " + item.title )

        # Copy the related items for this user (if specified)
        if relationships:
            for folder in folders:
                folder_items = user_content[folder]
                for item in folder_items:
                    try:
                        copy_relationships(target, copied_items, item, 
                                           relationships, user, folder)
                    except:
                        print("Error setting relationship for: " + item.title)


**************
adams.powell


<Item title:"AR" type:Feature Service owner:brown.rogers>

<Item title:"AR" type:CSV owner:brown.rogers>

<Item title:"Brown Rogers response locations" type:Web Map owner:brown.rogers>

**************
allen.price


...

<Item title:"Brown Rogers response locations" type:Web Map owner:brown.rogers>

**************
brown.rogers


<Item title:"AR" type:Feature Service owner:brown.rogers>

<Item title:"AR" type:CSV owner:brown.rogers>

<Item title:"Brown Rogers response locations" type:Web Map owner:brown.rogers>

In [31]:
AR = target.content.search("AR", item_type='Feature Layer')[0]

In [32]:
AR.url

'http://Dev003327.esri.com/server/rest/services/Hosted/AR/FeatureServer'